In [1]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 80.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 67.6 MB/s eta 0:00:0000:0100:01


In [2]:
import pandas as pd 
from sodapy import Socrata

In [3]:
client = Socrata("data.sfgov.org", None)

In [4]:
results = client.get("wg3w-h783", limit=1000, offset= 0)

In [83]:
condition = """
report_datetime >= '2023-01-01T00:01:00.000' 
AND
report_datetime <= '2023-01-02T00:01:00.000'
"""

In [5]:
# This would create offset.txt file to keep record of # processed rows 

offset = ''
try:
    with open("offset.txt", "r") as f:
        offset = f.readline()
except FileNotFoundError:
    with open("offset.txt", "w+") as f:
        f.write("0")
        f.seek(0)
        offset = f.readline()
        
offset = int(offset)
offset

0

In [6]:
records = client.get("wg3w-h783", select="count(*)")

In [7]:
total_no_of_records = records[0]['count']

In [8]:
total_no_of_records = int(total_no_of_records)
total_no_of_records

713737

In [9]:
blob = pd.DataFrame()

In [10]:
while total_no_of_records != offset: 
    results = client.get("wg3w-h783", offset=offset, limit=50000)
    results_df = pd.DataFrame.from_records(results)

    # if blob.empty: 
    #     blob = results_df
    # else: 
    #     blob = blob.append(results_df, ignore_index=True)

    blob = pd.concat([results_df, blob])

    offset += len(results_df)
    offset

    with open("offset.txt", "w+") as f: 
        f.write(str(offset))

In [15]:
blob.shape

(713737, 34)

In [16]:
blob.head(1000).describe()

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,report_type_code,...,latitude,longitude,point,:@computed_region_jwn9_ihcz,:@computed_region_h4ep_8xdi,:@computed_region_26cr_cadq,:@computed_region_qgnn_b9vv,:@computed_region_n4xg_c4py,:@computed_region_nqbw_i6c3,:@computed_region_jg9y_a9du
count,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,...,558,558,558,550,56,558,558,60,109,6
unique,978,545,426,6,7,990,1000,999,997,4,...,465,465,465,97,1,11,10,1,5,1
top,2021-11-23T13:00:00.000,2021-11-23T00:00:00.000,00:00,2021,Saturday,2021-11-23T13:00:00.000,108341406374,1057398,210497265,II,...,37.7751608100771,-122.40363551943442,"{'type': 'Point', 'coordinates': [-122.4036355...",32,1,10,6,1,3,1
freq,6,18,39,569,154,6,1,2,2,728,...,7,7,7,50,56,113,88,60,49,6


In [17]:
blob.head(4)

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,report_type_code,...,latitude,longitude,point,:@computed_region_jwn9_ihcz,:@computed_region_h4ep_8xdi,:@computed_region_26cr_cadq,:@computed_region_qgnn_b9vv,:@computed_region_n4xg_c4py,:@computed_region_nqbw_i6c3,:@computed_region_jg9y_a9du
0,2021-08-21T19:00:00.000,2021-08-21T00:00:00.000,19:00,2021,Saturday,2021-08-31T11:04:00.000,108341406374,1083414,216155465,II,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-10-06T21:57:00.000,2021-10-06T00:00:00.000,21:57,2021,Wednesday,2021-10-06T21:57:00.000,107789907021,1077899,210652237,VI,...,37.78640961281089,-122.40803623744476,"{'type': 'Point', 'coordinates': [-122.4080362...",19,1,3,6,NaN,NaN,NaN
2,2018-04-25T20:00:00.000,2018-04-25T00:00:00.000,20:00,2018,Wednesday,2018-04-28T13:50:00.000,66955909320,669559,186111470,II,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-12-02T11:05:00.000,2019-12-02T00:00:00.000,11:05,2019,Monday,2019-12-04T15:40:00.000,87813806244,878138,196260835,II,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
from pathlib import Path

In [28]:
# Extract year, month, and date from the incident_date column
blob['year'] = pd.DatetimeIndex(blob['incident_date']).year.astype(str)
blob['month'] = pd.DatetimeIndex(blob['incident_date']).month.astype(str).str.zfill(2)
blob['day'] = pd.DatetimeIndex(blob['incident_date']).day.astype(str).str.zfill(2)

In [29]:
# Create directory structure if it doesn't exist
for year in blob['year'].unique():
    year_dir = Path('data') / year
    year_dir.mkdir(parents=True, exist_ok=True)
    for month in blob[blob['year'] == year]['month'].unique():
        month_dir = year_dir / month
        month_dir.mkdir(parents=True, exist_ok=True)

In [32]:
# Save the DataFrame into year/month/day CSV files
for _, row in blob.iterrows():
    file_path = Path('data') / row['year'] / row['month'] / f"{row['day']}.csv"
    row.to_frame().T.to_csv(file_path, mode='a', header=not file_path.exists(), index=False)

KeyboardInterrupt: 

In [82]:
results_df = pd.DataFrame.from_records(results)

In [80]:
with open("offset.txt", "w+") as f: 
    f.write(str(offset))

In [88]:
results = client.get("wg3w-h783", offset=offset,order="report_datetime")

In [84]:
results = client.get("wg3w-h783", where=condition)

In [90]:
results_df

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,cad_number,...,resolution,intersection,cnn,police_district,analysis_neighborhood,supervisor_district,supervisor_district_2012,latitude,longitude,point
0,2023-03-30T18:00:00.000,2023-03-30T00:00:00.000,18:00,2023,Thursday,2023-04-02T21:10:00.000,126170906241,1261709,230229076,230922935,...,Open or Active,16TH ST \ JOSE SARRIA CT \ POND ST,25800000,Mission,Castro/Upper Market,8,8,37.76428896385761,-122.43227951733411,"{'type': 'Point', 'coordinates': [-122.4322795..."
1,2023-04-02T14:25:00.000,2023-04-02T00:00:00.000,14:25,2023,Sunday,2023-04-02T21:35:00.000,126171128161,1261711,230226486,230922904,...,Open or Active,PALOU AVE \ HAWES ST,20118000,Bayview,Bayview Hunters Point,10,10,37.72867801184137,-122.38156441612668,"{'type': 'Point', 'coordinates': [-122.3815644..."
2,2023-04-02T14:25:00.000,2023-04-02T00:00:00.000,14:25,2023,Sunday,2023-04-02T21:35:00.000,126171128160,1261711,230226486,230922904,...,Open or Active,PALOU AVE \ HAWES ST,20118000,Bayview,Bayview Hunters Point,10,10,37.72867801184137,-122.38156441612668,"{'type': 'Point', 'coordinates': [-122.3815644..."
3,2023-04-02T21:37:00.000,2023-04-02T00:00:00.000,21:37,2023,Sunday,2023-04-02T21:37:00.000,126171971012,1261719,230216631,230872414,...,Open or Active,BRYANT ST \ LANGTON ST,23876000,Southern,South of Market,6,6,37.77385630397913,-122.40528580238285,"{'type': 'Point', 'coordinates': [-122.4052858..."
4,2023-04-02T21:38:00.000,2023-04-02T00:00:00.000,21:38,2023,Sunday,2023-04-02T21:38:00.000,126171707043,1261717,230125004,NaN,...,Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN
5,2023-04-02T22:35:00.000,2023-04-02T00:00:00.000,22:35,2023,Sunday,2023-04-02T22:35:00.000,126173007055,1261730,230229195,230923119,...,Open or Active,04TH ST \ BRANNAN ST,23827000,Southern,South of Market,6,6,37.7783268304671,-122.39653063923612,"{'type': 'Point', 'coordinates': [-122.3965306..."
6,2023-04-02T22:41:00.000,2023-04-02T00:00:00.000,22:41,2023,Sunday,2023-04-02T22:41:00.000,126141264020,1261412,230227020,230913295,...,Open or Active,ARBALLO DR \ SERRANO DR,23098000,Taraval,Lakeshore,7,7,37.719109027120155,-122.48310424113126,"{'type': 'Point', 'coordinates': [-122.4831042..."


Below things should be considered when pushing to gcs in parquet

In [28]:
results_filtered = results_df[results_df[':@computed_region_nqbw_i6c3'].notnull()]

In [38]:
results_filtered.head(5)

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,report_type_code,...,supervisor_district_2012,latitude,longitude,point,:@computed_region_26cr_cadq,:@computed_region_qgnn_b9vv,:@computed_region_jwn9_ihcz,:@computed_region_h4ep_8xdi,:@computed_region_nqbw_i6c3,:@computed_region_n4xg_c4py
220,2021-07-12 18:18:00,2021-07-12,18:18:00,2021,Monday,2021-07-12T18:26:00.000,104984326080,1049843,210440565,II,...,9,37.74817939136343,-122.41917060471224,"{'type': 'Point', 'coordinates': [-122.4191706...",2,3,53,NaN,3,NaN
221,2021-07-07 08:18:00,2021-07-07,08:18:00,2021,Wednesday,2021-07-07T08:28:00.000,104785804134,1047858,210426383,II,...,9,37.753836586731786,-122.41859394629981,"{'type': 'Point', 'coordinates': [-122.4185939...",2,3,53,NaN,3,NaN
304,2021-07-12 12:20:00,2021-07-12,12:20:00,2021,Monday,2021-07-12T15:20:00.000,104980919057,1049809,210440703,II,...,6,37.78321431177312,-122.41076482950653,"{'type': 'Point', 'coordinates': [-122.4107648...",10,5,20,1,1,1
309,2019-06-11 16:30:00,2019-06-11,16:30:00,2019,Tuesday,2021-06-05T05:17:00.000,103770002004,1037700,210348157,II,...,6,37.775953228502786,-122.40884575063417,"{'type': 'Point', 'coordinates': [-122.4088457...",10,1,32,NaN,1,1
342,2022-09-25 17:00:00,2022-09-25,17:00:00,2022,Sunday,2022-09-26T17:09:00.000,119741374000,1197413,220659671,II,...,9,37.75723240847821,-122.41562619847495,"{'type': 'Point', 'coordinates': [-122.4156261...",2,3,53,NaN,3,NaN


In [33]:
results_filtered['incident_datetime'] = results_filtered['incident_datetime'].astype('datetime64[ns]')

/tmp/ipykernel_11518/2783132438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_filtered['incident_datetime'] = results_filtered['incident_datetime'].astype('datetime64[ns]')


In [34]:
results_filtered['incident_date'] = results_filtered['incident_date'].astype('datetime64[ns]')

/tmp/ipykernel_11518/4139641677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_filtered['incident_date'] = results_filtered['incident_date'].astype('datetime64[ns]')


In [35]:
results_filtered['incident_time'] = pd.to_datetime(results_filtered['incident_time']).dt.time

TypeError: <class 'datetime.time'> is not convertible to datetime, at position 0

In [40]:
results_filtered['incident_year'] = results_filtered['incident_year'].astype(int) 

/tmp/ipykernel_11518/762521036.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_filtered['incident_year'] = results_filtered['incident_year'].astype(int)


In [43]:
results_filtered.to_parquet('test.parquet')

In [44]:
par_df = pd.read_parquet('test.parquet')

In [47]:
par_df.dtypes

incident_datetime              datetime64[ns]
incident_date                  datetime64[ns]
incident_time                          object
incident_year                           int64
incident_day_of_week                   object
report_datetime                        object
row_id                                 object
incident_id                            object
incident_number                        object
report_type_code                       object
report_type_description                object
incident_code                          object
incident_category                      object
incident_subcategory                   object
incident_description                   object
resolution                             object
police_district                        object
filed_online                           object
cad_number                             object
intersection                           object
cnn                                    object
analysis_neighborhood             